In [1]:
import yfinance as yf
import numpy as np
from arch import arch_model
from datetime import datetime
import matplotlib.pyplot as plt
import data_preprocessing
# Download data with yfinance
ticker = 'LREN3.SA'
start_date = '2020-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

data = yf.download(ticker, start=start_date, end=end_date)
data.reset_index(inplace=True)
data['log_return'] = np.log(data['Close'] / data['Close'].shift(1))
data = data.dropna()

ModuleNotFoundError: No module named 'data_preprocessing'

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(data['log_return'], lags=40)
plot_pacf(data['log_return'], lags=40)
plt.show()

In [ ]:
# Fit GARCH Model
train_size = int(0.75 * len(data))
train_set = data.loc[data.index[:train_size], "log_return"]
test_set = data.loc[data.index[train_size:], "log_return"]

am = arch_model(train_set, mean='Zero', vol='GARCH', p=40, q=40)
res = am.fit(disp='off')

print(res.summary())
# Forecast volatility
forecasts = res.forecast(horizon=len(data) - train_size)

In [ ]:
#plt.plot(forecasts.variance.iloc[-1, :])
var_forecast = forecasts.variance.T
var_forecast.set_index(test_set.index, inplace=True)
plt.plot(test_set)
plt.plot(var_forecast)
plt.show()